In [1]:
import numpy as np
import numba

import rotations
import test_rotations

## Testing conversions between different representations

### Roundtrips between unit quaternions and rotation matrices

In [2]:
for i in range(1000):
    test_rotations.test_matrix_quaternion()

In [3]:
for i in range(1000):
    test_rotations.test_quaternion_matrix()

### Roundtrips between unit quaternions and Euler vectors

In [4]:
for i in range(1000):
    test_rotations.test_quaternion_euler()

In [5]:
for i in range(1000):
    test_rotations.test_euler_quaternion()
    
# Note: Only works when Euler vector has -2pi < theta < +2pi

known_euler [ 2.8869021  -1.27821708  0.79345442]
euler [-2.68505682  1.1888472  -0.73797799]


AssertionError: 

### Full roundtrips between rotation matrices and Euler vectors

In [7]:
for i in range(1000):
    test_rotations.test_matrix_euler()

In [6]:
for i in range(1000):
    test_rotations.test_euler_matrix()

known_euler [-0.70713805 -1.25724213  2.99799183]
euler [ 0.62833955  1.11714391 -2.66391669]


AssertionError: 

### Other tests

In [8]:
test_rotations.test_quaternion_from_euler()

In [9]:
for i in range(1000):
    test_rotations.test_matrix_from_euler()

In [7]:
test_rotations.test_update_euler()

## Rotations greater than pi are broken!!!

In [10]:
def roundtrip(euler):
    
    euler = np.copy(euler)
    
    rev = int(np.linalg.norm(euler) / np.pi)
    print("rev", rev)
    
    if rev > 0:
        euler /= rev*np.pi
    
    quaternion = rotations.quaternion_from_euler(euler)

    matrix = rotations.matrix_from_quaternion(quaternion)
    
    quaternion2 = rotations.quaternion_from_matrix(matrix)
    
    assert np.allclose(quaternion, quaternion2)
    
    euler2 = rotations.euler_from_quaternion(quaternion2)
    
    if rev>0:
        euler2 *= rev*np.pi
    
    return euler2

In [11]:
def test_euler_matrix():
    while True:
        known_euler = np.random.normal(loc=0.0, scale=2*np.pi, size=3)
        if np.linalg.norm(known_euler) < 5*np.pi:
            break

    euler = roundtrip(known_euler)

    check = np.allclose(known_euler, euler)

    if not check:
        print("known_euler", known_euler)
        print("euler", euler)

    assert check

In [32]:
test_euler_matrix()

rev 3


## Interpolation of rotations

In [33]:
def random_euler_and_SO3():
    # create a random rotation in axis-angle representation
    while True:
        euler = np.random.normal(loc=0.0, scale=1.0, size=3)
        if np.linalg.norm(euler) < 1*np.pi:
            break

    # convert to a matrix
    matrix = rotations.matrix_from_quaternion(rotations.quaternion_from_euler(euler))
    
    assert np.allclose(np.dot(matrix, matrix.T), np.eye(3))
    
    return euler, matrix

In [34]:
def interpolate_euler(euler_left, euler_right):
    return (euler_left + euler_right) / 2

In [35]:
#@numba.jit(numba.float64[:, :](numba.float64[:, :], numba.float64[:, :]), nopython=True)
def interpolate_matrix(matrix_left, matrix_right):
    Qd = np.dot(matrix_right, matrix_left.T)
    ed = rotations.euler_from_quaternion(rotations.quaternion_from_matrix(Qd))

    ed2 = ed / 2    
    Qd2 = rotations.matrix_from_quaternion(rotations.quaternion_from_euler(ed2))
    
    Qi = np.dot(Qd2, matrix_left)
    return Qi

In [36]:
def experiment():
    el, Ql = random_euler_and_SO3()
    er, Qr = random_euler_and_SO3()
    
    ei_raw = interpolate_euler(el, er)
    Qi_raw = rotations.matrix_from_quaternion(rotations.quaternion_from_euler(ei_raw))
    
    Qi = interpolate_matrix(Ql, Qr)
    
    rel_error = np.linalg.norm(Qi_raw - Qi) / np.linalg.norm(Qi)
    
    print(rel_error * 100, 'percent')

In [54]:
experiment()

11.989958256 percent


In [55]:
def SO3_from_theta(theta):
    matrix = np.array([[+np.cos(theta), -np.sin(theta), 0],
                       [+np.sin(theta), +np.cos(theta), 0],
                       [0, 0, 1]])
    return matrix

In [56]:
Ql = SO3_from_theta(2)
Qr = SO3_from_theta(4)
Qi = interpolate_matrix(Ql, Qr)
Qi

array([[-0.9899925 , -0.14112001,  0.        ],
       [ 0.14112001, -0.9899925 ,  0.        ],
       [ 0.        ,  0.        ,  1.        ]])

In [57]:
SO3_from_theta(3)

array([[-0.9899925 , -0.14112001,  0.        ],
       [ 0.14112001, -0.9899925 ,  0.        ],
       [ 0.        ,  0.        ,  1.        ]])

In [60]:
el, Ql = random_euler_and_SO3()
er, Qr = random_euler_and_SO3()

In [61]:
%timeit interpolate_euler(el, er)

1.95 µs ± 18.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [62]:
%timeit interpolate_matrix(Ql, Qr)

12.5 µs ± 507 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
